In [1]:
""" Calcuates the next input for black box optimiation function using
a Gaussian Process surrogate with Matern kernel and Expected Improvement
acquisition function.

Change the nu value of the Matern kernel to make it rougher or smoother
Larger nu means smoother kernel. Smaller nu means rougher kernel.

Change the xi value of the acquisition function to make it more or less
exploratory.
Larger xi means more exploratory. Smaller xi means less exploratory (going
towards exploitation).

Change history:
v0.01 - 05/01/2025 - First version, refer description above.

"""
# Import Libraries

import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel as C, WhiteKernel
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
from scipy.optimize import minimize
from datetime import datetime


In [3]:

# ------------------------------------------------------------
# Load initial data
# ------------------------------------------------------------
X = np.load("data/fn1_initial_inputs.npy")        # shape (n0, d)
y = np.load("data/fn1_initial_outputs.npy")       # shape (n0,)

today_str = datetime.today().strftime('%Y%m%d')
output_filename = "fn1_next_input_" + today_str + ".npy"



In [4]:
# ------------------------------------------------------------
# Normalise inputs and outputs
# ------------------------------------------------------------
x_scaler = StandardScaler()
Xn = x_scaler.fit_transform(X)

y_mean = y.mean()
y_std = y.std() if y.std() > 0 else 1.0
yn = (y - y_mean) / y_std                 # GP works better with normalized target

# ------------------------------------------------------------
# Fit a Gaussian Process surrogate
# ------------------------------------------------------------
d = X.shape[1]
kernel = C(1.0, (1e-3, 1e3)) * Matern(length_scale=np.ones(d),
                                      length_scale_bounds=(1e-2, 1e2),
                                      nu=2.5)
kernel += WhiteKernel(noise_level=1e-6,
                      noise_level_bounds=(1e-10, 1e1))

gp = GaussianProcessRegressor(kernel=kernel,
                              normalize_y=False,
                              n_restarts_optimizer=10,
                              random_state=0)
gp.fit(Xn, yn)



/usr/local/lib/python3.12/dist-packages/sklearn/gaussian_process/kernels.py:452: ConvergenceWarning: The optimal value found for dimension 1 of parameter k1__k2__length_scale is close to the specified upper bound 100.0. Increasing the bound and calling fit again may find a better value.
  warnings.warn(


GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=[1, 1], nu=2.5) + WhiteKernel(noise_level=1e-06),
                         n_restarts_optimizer=10, random_state=0)

In [5]:
# ------------------------------------------------------------
# Define Expected Improvement acquisition (for maximization)
# ------------------------------------------------------------
f_best = yn.max()
xi = 0.01     # exploration parameter

def predict_raw(x_raw):
    """Predict mean and std in normalized GP space for a raw input x_raw."""
    x = np.atleast_2d(x_raw)
    xn = x_scaler.transform(x)
    mu, sigma = gp.predict(xn, return_std=True)
    return mu.ravel(), sigma.ravel()

def expected_improvement_raw(x_raw, xi=xi):
    """Compute EI for a raw input x_raw."""
    mu, sigma = predict_raw(x_raw)
    sigma = np.maximum(sigma, 1e-9)       # avoid division by zero
    z = (mu - f_best - xi) / sigma
    ei = (mu - f_best - xi) * norm.cdf(z) + sigma * norm.pdf(z)
    return ei.ravel()[0]



In [6]:
# ------------------------------------------------------------
# Search for the next query point
# ------------------------------------------------------------
# Domain is assumed to be [0,1]^d (based on inspection of initial data)
bounds = [(0.0, 1.0)] * d

# Global random search to find good seeds
n_seeds = 2000
rng = np.random.default_rng(1)
candidates = rng.uniform(0.0, 1.0, size=(n_seeds, d))
ei_vals = np.array([expected_improvement_raw(c) for c in candidates])
best_idx = np.argmax(ei_vals)

# Take top few seeds for local optimisation
seed_points = candidates[np.argsort(-ei_vals)[:20]]

best_x = None
best_val = -1.0
for s in seed_points:
    res = minimize(lambda xx: -expected_improvement_raw(xx),
                   x0=s,
                   bounds=bounds,
                   method="L-BFGS-B",
                   options={'maxiter':300})
    if res.success:
        val = -res.fun
        if val > best_val:
            best_val = val
            best_x = res.x.copy()

# Fallback if optimizer fails
if best_x is None:
    best_x = candidates[best_idx]
    best_val = ei_vals[best_idx]

# Meet capstone requirement
epsilon = 1e-6
best_x = np.clip(best_x, 0.0, 1.0 - epsilon)     # ensure in [0,1] i.e begins with 0
best_x = np.round(best_x, 6)           # specified to 6 decimals



In [7]:
# ------------------------------------------------------------
# Report results
# ------------------------------------------------------------
suggested_point = np.atleast_1d(best_x)
mu_s, sigma_s = predict_raw(suggested_point.reshape(1, -1))

# Convert mean/std back to original y-scale
mu_orig = mu_s * y_std + y_mean
sigma_orig = sigma_s * y_std

print("Suggested next query point (raw input space):", suggested_point)
print("Expected Improvement at this point (normalized):", best_val)
print("GP predicted mean at this point (original y scale):", mu_orig)
print("GP predicted stddev at this point (original y scale):", sigma_orig)

# Optionally save the suggestion
np.save(output_filename, suggested_point)


Suggested next query point (raw input space): [0.701438 0.897524]
Expected Improvement at this point (normalized): 0.3496858120420145
GP predicted mean at this point (original y scale): [1.35307863e-05]
GP predicted stddev at this point (original y scale): [0.00094485]
